In [1]:
import os, pathlib, numpy as np, matplotlib.pyplot as plt, warnings
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

2025-09-26 09:15:36.633524: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758878136.971211      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758878137.060069      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [25]:
#tf.config.optimizer.set_experimental_options({'layout_optimizer': False})

### ***Constants & File Path***

In [3]:
SEED = 58
IMG_SIZE = (160, 160)
BATCH_SIZE = 32
DATA_DIR = "/kaggle/input/combined-forest-fire/combined_data/train"
TEST_DIR = "/kaggle/input/combined-forest-fire/combined_data/test"
VAL_SPLIT = 0.2
AUTOTUNE = tf.data.AUTOTUNE

tf.random.set_seed(SEED)
np.random.seed(SEED)

### ***Dataset Creation (train/val split)***

In [4]:
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import experimental as tfdata_exp

train_ds = image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    color_mode="rgb"
)
val_ds = image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    color_mode="rgb"
)

class_names = train_ds.class_names

test_ds = image_dataset_from_directory(
    TEST_DIR,
    image_size=IMG_SIZE, batch_size=BATCH_SIZE, color_mode="rgb",
    shuffle=False,               
    class_names=class_names        
)

NUM_CLASSES = len(class_names)
print("classes:", class_names)

train_ds = train_ds.apply(tfdata_exp.ignore_errors())
val_ds = val_ds.apply(tfdata_exp.ignore_errors())
test_ds = test_ds.apply(tfdata_exp.ignore_errors())

Found 20620 files belonging to 2 classes.
Using 16496 files for training.


I0000 00:00:1758789846.547760      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1758789846.548581      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Found 20620 files belonging to 2 classes.
Using 4124 files for validation.
Found 2935 files belonging to 2 classes.
classes: ['fire', 'nonfire']


### ***Optimization & Data Augmentation***

In [5]:
from tensorflow import keras
from tensorflow.keras import layers

train_ds = train_ds.shuffle(1024).prefetch(AUTOTUNE)
val_ds   = val_ds.prefetch(AUTOTUNE)

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
], name="augmentation")

### ***Building Baseline***


In [7]:
def build_baseline(input_shape=IMG_SIZE+(3,), num_classes=NUM_CLASSES):
    inputs = layers.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = layers.Rescaling(1./255)(x)

    x = layers.Conv2D(32, 3, padding="same", activation="relu")(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(64, 3, padding="same", activation="relu")(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(128, 3, padding="same", activation="relu")(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Dropout(0.3)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = keras.Model(inputs, outputs, name="fire_predict")
    return model

model = build_baseline()
model.summary()

Model: "fire_predict"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ augmentation (Sequential)       │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_1 (Rescaling)         │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 160, 160, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 80, 80, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 80, 80, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 40, 40, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 40, 40, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 51200)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     6,553,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,647,234 (25.36 MB)

 Trainable params: 6,647,234 (25.36 MB)

 Non-trainable params: 0 (0.00 B)

### ***Compiling, Callbacks and Model Training***

In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor="val_accuracy"),
    keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_loss"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(train_ds, validation_data=val_ds, epochs=15,callbacks=callbacks)

model.evaluate(test_ds, verbose=1)

Epoch 1/15


E0000 00:00:1758789905.966745      36 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/fire_predict_1/dropout_2_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
I0000 00:00:1758790037.662601     103 cuda_dnn.cc:529] Loaded cuDNN version 90300


    279/Unknown 149s 41ms/step - accuracy: 0.8501 - loss: 0.3710

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


279/279 ━━━━━━━━━━━━━━━━━━━━ 182s 160ms/step - accuracy: 0.8502 - loss: 0.3706 - val_accuracy: 0.9271 - val_loss: 0.1598 - learning_rate: 0.0010
Epoch 2/15


Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG


279/279 ━━━━━━━━━━━━━━━━━━━━ 148s 139ms/step - accuracy: 0.9267 - loss: 0.1754 - val_accuracy: 0.9412 - val_loss: 0.1323 - learning_rate: 0.0010
Epoch 3/15


Invalid SOS parameters for sequential JPEG


261/279 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9375 - loss: 0.1578

### ***Model Saving (if needed)***

In [ ]:
model.save("/kaggle/working/my_model.h5")
#model.save_weights("/kaggle/working/my_model.weights.h5")

### ***Predict a Single Image (with Top-1 Probability)***

In [ ]:
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np, pathlib

def predict_image(img_path, model, img_size=IMG_SIZE, class_names=class_names):
    # Güvenli RGB + yeniden boyutlandırma
    img = load_img(img_path, target_size=img_size, color_mode="rgb")
    arr = img_to_array(img)              # (H,W,3), dtype=float32, 0..255
    arr = np.expand_dims(arr, axis=0)    # (1,H,W,3)

    # Tahmin
    probs = model.predict(arr, verbose=0)[0]          # (num_classes,)
    idx   = int(np.argmax(probs))
    pred  = class_names[idx]
    conf  = float(probs[idx])

    return {"path": str(img_path), "pred": pred, "conf": conf, "probs": probs}

TEST_IMG = "/kaggle/input/testtest/testdata/nonfiretest/NF_9.jpg"
res = predict_image(TEST_IMG, model)
print(f"Pred: {res['pred']}  |  conf: {res['conf']:.3f}")

In [ ]:
model.save_weights("/kaggle/working/fire_nonfire.weights.h5")

In [ ]:
import glob, os
from PIL import Image, UnidentifiedImageError

def predict_folder(folder_path, model, img_size=IMG_SIZE, class_names=class_names):
    exts = ("*.jpg","*.jpeg","*.png","*.bmp","*.webp")
    files = []
    for e in exts:
        files.extend(glob.glob(os.path.join(folder_path, e)))
    results, bad = [], []

    for p in files:
        try:
                # First, quickly check if it can be opened and converted to RGB (skip corrupted ones)            with Image.open(p) as im:
                im.convert("RGB")
            out = predict_image(p, model, img_size, class_names)
            results.append(out)
        except (UnidentifiedImageError, OSError):
            bad.append(p)

    ## a small summary table
    import pandas as pd
    if results:
        df = pd.DataFrame([{"path": r["path"], "pred": r["pred"], "conf": r["conf"]} for r in results])
        display(df.sort_values("conf", ascending=False).reset_index(drop=True))
        print(f"\nTotal: {len(results)} Image predicted. Skipped (corrupted): {len(bad)}")
    else:
        print("No valid images found. ")
    if bad:
        print("Skipped (corrupted) samples (first 10):")
        for p in bad[:10]:
            print(" -", p)

TEST_DIR = "/kaggle/input/testtest/testdata/firetest" 
predict_folder(TEST_DIR, model)